#### import the things

In [1]:
import pandas as pd
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
idx = pd.IndexSlice



In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline, Pipeline

In [7]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [105]:
from sklearn import metrics

In [16]:
X_train = pd.read_pickle(os.path.join(data_fol, "combined_model", "__X_train.pkl"))
X_test = pd.read_pickle(os.path.join(data_fol, "combined_model", "__X_test.pkl"))
y_train = pd.read_pickle(os.path.join(data_fol, "combined_model", "__y_train.pkl"))
y_test = pd.read_pickle(os.path.join(data_fol, "combined_model", "__y_test.pkl"))

In [17]:
X_train

,user_followers_count,user_verified,media_count,media_animated_gif,media_photo,media_video,hashtags_clean,text_clean
tid,,,,,,,,
1361890179951296512,360,0,0,0,0,0,,dellamorte late september clip going around...
1358354354462289920,382,0,0,0,0,0,,know lot poz ppl even know many people ppl ...
1363916379519336448,417,0,0,0,0,0,,hiv covid pandemic show u fear disease coloure...
1369296990782320648,6656,0,0,0,0,0,uequalsu,tomorrow wednesday march th pm et join tweetst...
1369559005811662851,104,0,0,0,0,0,,next generation hiv drug looking really promis...
...,...,...,...,...,...,...,...,...
1369950382693232643,6045,0,0,0,0,0,,want part multistakeholder task force highlev...
1368967563837857792,15198,1,0,0,0,0,,announced year collaboration radbouduni provi...
1365011079600824322,416,0,0,0,0,0,,cjtvguy diamondsstart nurse ball aid hiv benef...


#### try a few different transformations

Code adapted from https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

In [18]:
text = X_train['text_clean']

In [24]:
n_features = 100
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')


tfidf = tfidf_vectorizer.fit_transform(text)


tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(text)

In [25]:
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names())

In [26]:
tfidf_df.head()

,africa,aid,awareness,black,blood,cancer,care,cause,child,come,...,treatment,use,vaccine,virus,want,way,woman,work,world,year
0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.52839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.417984,0.0,0.0,0.0
4,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [27]:
tfidf_test = tfidf_vectorizer.transform(X_test['text_clean'])

In [28]:
tfidf_test_df = pd.DataFrame(tfidf_test.toarray(), columns=tfidf_vectorizer.get_feature_names())

#### Optimize NMF

In [106]:
def get_score(model, data, scorer=metrics.explained_variance_score):
    """ Estimate performance of the model on the data 
    function from https://stackoverflow.com/questions/48148689/how-to-compare-predictive-power-of-pca-and-nmf"""
    prediction = model.inverse_transform(model.transform(data))
    return scorer(data, prediction)

In [126]:
#create temp train/test split to evaluate NMF n_components
tfidf_test_ind = random.sample(range(len(tfidf_df)), k=int(len(tfidf_df)*.2))

tfidf_train = tfidf_df.drop(index=tfidf_test_ind)
tfidf_test = tfidf_df.iloc[tfidf_test_ind]

tfidf_train.shape, tfidf_test.shape

((9608, 100), (2401, 100))

In [124]:
#code adapated from same link as get_score function above
ks = [5, 10, 30, 50, 80, 90]
performance = []
for k in ks:
    nmf = NMF(n_components=k, init='random', random_state=1).fit(tfidf_train)
    train = get_score(nmf, tfidf_train)
    test = get_score(nmf, tfidf_test)
    performance.append(("for k={}, train explained variance is {} and test explained variance is {}".format(k, train, test)))    
    print('boom')


boom
boom
boom
boom
boom
boom


In [125]:
performance

['for k=5, train explained variance is 0.04965222899504065 and test explained variance is 0.04854808037781701',
 'for k=10, train explained variance is 0.10896900071835125 and test explained variance is 0.10726659797169909',
 'for k=30, train explained variance is 0.3231908147694511 and test explained variance is 0.3195816958712332',
 'for k=50, train explained variance is 0.5264733890853058 and test explained variance is 0.5225931094785763',
 'for k=80, train explained variance is 0.816785492965659 and test explained variance is 0.8147271110699765',
 'for k=90, train explained variance is 0.9135813727945948 and test explained variance is 0.9121370562701634']

to avoid overfitting, I will go with n_components=80. 

In [127]:
nmf = NMF(n_components=80, init='random', random_state=1)

In [128]:
tfidf_nmf = nmf.fit_transform(tfidf_df)
tfidf_nmf_test = nmf.transform(tfidf_test_df)

In [148]:
tfidf_nmf_df = pd.DataFrame(tfidf_nmf, columns=range(tfidf_nmf.shape[1]), index=X_train.index)
tfidf_nmf_test_df = pd.DataFrame(tfidf_nmf_test, columns=range(tfidf_nmf_test.shape[1]), index=X_test.index)

In [150]:
fname = "X_train_tfidf_nmf.pkl"
pd.to_pickle(tfidf_nmf_df, os.path.join(data_fol, "combined_model", fname))

In [151]:
fname = "X_test_tfidf_nmf.pkl"
pd.to_pickle(tfidf_nmf_test_df, os.path.join(data_fol, "combined_model", fname))